# Facebook Freunde Scrapen
- Eine super Datenquelle um Netzwerke zu verstehen sind die eigenen Freunde. 
- Diese lassen sich von Facebook scrapen, da Facebook natürlich keinen Export erlaubt - d'oh. 

In [20]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json
import csv
import re

In [21]:
s = webdriver.Chrome('/usr/local/bin/chromedriver')

https://selenium-python.readthedocs.io/api.html#selenium.webdriver.remote.webdriver.WebDriver.find_element_by_link_text- 

In [3]:
USERNAME = "edzard.schade@htwchur.ch"
PASSWORD = "CHUR2010"
MY_SLUG = "profile.php?id=100008456893753"

In [13]:
def login(s):
    try:
        print("Logging into FB")
        s.get("http://facebook.com/login")
        s.find_element_by_id("email").send_keys(USERNAME)
        r = s.find_element_by_id("pass")
        r.send_keys(PASSWORD)
        r.send_keys(Keys.RETURN)
    except:
        print("Probably already logged in...")

In [22]:
def scrape_friends(s):
    login(s)
    #Scrape my friends
    #s.get("https://www.facebook.com/%s/friends" % MY_SLUG)
    s.get("https://www.facebook.com/profile.php?id=100008456893753&sk=friends&ft_ref=flsa")
    for i in range(30):
        print("scrolling...and...waiting..%s" % i)
        s.execute_script('window.scroll(0,100000);')
        time.sleep(1)
    # get html
    elem = s.find_element_by_xpath("//*")
    source_code = elem.get_attribute("outerHTML")
    soup = BeautifulSoup(source_code, 'lxml')
    friends = soup.select(".fcb a")
    print("Scraping {%s} friends" % len(friends))
    with open('friends.csv', mode='w') as myfile:
        mycsv = csv.writer(myfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        mycsv.writerow(["Friend ID", "Friend Name"])
        for friend in friends:
          try:
            friend_name = friend.text.encode("ascii","ignore")
            friend_id = json.loads(friend["data-gt"])["engagement"]["eng_tid"].encode("ascii","ignore")
            mycsv.writerow([friend_id, friend_name])
          except:
            print("Problems obtaining %s" % friend)

In [23]:
login(s)

Logging into FB


In [24]:
scrape_friends(s)

Logging into FB
Probably already logged in...
scrolling...and...waiting..0
scrolling...and...waiting..1
scrolling...and...waiting..2
scrolling...and...waiting..3
scrolling...and...waiting..4
scrolling...and...waiting..5
scrolling...and...waiting..6
scrolling...and...waiting..7
scrolling...and...waiting..8
scrolling...and...waiting..9
scrolling...and...waiting..10
scrolling...and...waiting..11
scrolling...and...waiting..12
scrolling...and...waiting..13
scrolling...and...waiting..14
scrolling...and...waiting..15
scrolling...and...waiting..16
scrolling...and...waiting..17
scrolling...and...waiting..18
scrolling...and...waiting..19
scrolling...and...waiting..20
scrolling...and...waiting..21
scrolling...and...waiting..22
scrolling...and...waiting..23
scrolling...and...waiting..24
scrolling...and...waiting..25
scrolling...and...waiting..26
scrolling...and...waiting..27
scrolling...and...waiting..28
scrolling...and...waiting..29
Scraping {226} friends
Problems obtaining <a data-hovercard="/aj

In [25]:
def scrape_mutual_friends(s):
    login(s)
    edgelist = []
    friendslist = []
    
    #Get FriendsIDs
    with open('friends.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for i, row in enumerate(csv_reader):
            if i != 0:
                friendslist.append([row["Friend ID"][2:-1],row["Friend Name"][2:-1]])
                
    #Get my Id
    s.get("http://facebook.com/%s" % MY_SLUG)
    elem = s.find_element_by_xpath("//*")
    source_code = elem.get_attribute("outerHTML")
    soup = BeautifulSoup(source_code, 'lxml')
    my_id = re.search(r"profile_id=(\d*)", source_code).groups()[0]
    print("My ID: %s" % my_id)

    for friends in friendslist:
        print("Visiting friend: %s" % friends[1])
        s.get("https://www.facebook.com/browse/mutual_friends/?uid=%s&node=%s" % (my_id,friends[0]))
        elem = s.find_element_by_xpath("//*")
        source_code = elem.get_attribute("outerHTML")
        soup = BeautifulSoup(source_code, 'lxml')
        mutual_friends = soup.select(".fcb a")
        print("Found %s mutual friends" % len(mutual_friends))
        for friend in mutual_friends:
            edgelist.append([friends[0],json.loads(friend["data-gt"])["engagement"]["eng_tid"].encode("ascii","ignore")])

    with open('network.csv', mode='w') as myfile:
        mycsv = csv.writer(myfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        mycsv.writerow(["From","To"])
        for edge in edgelist:
            mycsv.writerow([edge[0],edge[1]])

In [26]:
scrape_mutual_friends(s)

Logging into FB
Probably already logged in...
My ID: 100008456893753
Visiting friend: Christine Steffen
Found 25 mutual friends
Visiting friend: Anna-K. Mayer
Found 10 mutual friends
Visiting friend: Meinrad Schade
Found 30 mutual friends
Visiting friend: Dietmar Schade
Found 9 mutual friends
Visiting friend: Christoph Stuehn
Found 18 mutual friends
Visiting friend: Birgit Schade
Found 8 mutual friends
Visiting friend: Philip Kbler
Found 9 mutual friends
Visiting friend: Marcel Gamma
Found 30 mutual friends
Visiting friend: Daniel Fleischmann
Found 18 mutual friends
Visiting friend: Esther Hrlimann
Found 30 mutual friends
Visiting friend: Meret Ernst
Found 12 mutual friends
Visiting friend: Robert Gassmann
Found 14 mutual friends
Visiting friend: Dorotea Simeon
Found 4 mutual friends
Visiting friend: Ina Mller
Found 10 mutual friends
Visiting friend: Hannes Schade
Found 8 mutual friends
Visiting friend: Philipp Kuntschik
Found 12 mutual friends
Visiting friend: Ingo Barkow
Found 9 mutu

Found 14 mutual friends
Visiting friend: Rolf Probala
Found 23 mutual friends
Visiting friend: Regula Michell
Found 22 mutual friends
Visiting friend: Elisabeth Joris
Found 22 mutual friends
Visiting friend: Udo Schaufelberger
Found 4 mutual friends
Visiting friend: Peter Jaeggi
Found 30 mutual friends
Visiting friend: Gabriela Christen
Found 28 mutual friends
Visiting friend: Bettina Bsser
Found 30 mutual friends
Visiting friend: Heiner Hug
Found 19 mutual friends
Visiting friend: Andreas Schefer
Found 30 mutual friends
Visiting friend: Rudolf Mumenthaler
Found 20 mutual friends
Visiting friend: Stefan Lnzlinger
Found 14 mutual friends
Visiting friend: Florence Gehrig
Found 6 mutual friends
Visiting friend: Dominique Strebel
Found 30 mutual friends
Visiting friend: Eric Facon
Found 30 mutual friends
Visiting friend: Filip Dingerkus
Found 19 mutual friends
Visiting friend: Eliane Maria Degonda
Found 6 mutual friends
Visiting friend: Mirjam Gilgen-Schade
Found 7 mutual friends
Visiting 

In [27]:
import networkx as nx

def create_user_network():
    friendslist = []
    edgelist = []
    
    with open('friends.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for i, row in enumerate(csv_reader):
            if i != 0:
                friendslist.append([row["Friend ID"],row["Friend Name"]])        

    with open('network.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for i, row in enumerate(csv_reader):
            if i != 0:
                edgelist.append([row["From"],row["To"]])
    
    G  = nx.Graph(name="Friends")
                            
    for friend in friendslist:
        G.add_node(friend[0],label=friend[1], size=1)

    for edge in edgelist:
        G.add_edge(edge[0],edge[1],weight=1)
   
    nx.write_gexf(G,"Friends.gexf")

In [28]:
create_user_network()